In [1]:
from image_processing import *
import dlib
import cv2
import numpy as np

# Load the detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
import cv2

def enhance_image_resolution(image):
    # Load the super-resolution model
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"  # Change to the path of the model
    sr.readModel(path)
    sr.setModel("edsr", 4)  # You can change the model and scale as needed

    # Enhance the resolution of the image
    enhanced_image = sr.upsample(image)
    return enhanced_image


In [3]:
def detect_pupil(eye_image):
    # Enhance resolution
    eye_image = enhance_image_resolution(eye_image)

    # Preprocessing
    gray = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # Histogram Equalization
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Gaussian Blur

    # Edge detection
    edged = cv2.Canny(blurred, 50, 100)

    # Find contours
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and filter
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

    for contour in contours:
        # Approximate the contour
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * peri, True)

        # The pupil will be the first contour that is sufficiently circular
        area = cv2.contourArea(contour)
        if area > 30:  # You may need to adjust this threshold
            print(area)
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                cv2.circle(eye_image, (cX, cY), 7, (255, 255, 255), -1)
                cv2.imshow("Eye", eye_image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                return (cX, cY), contour

    return None, None


In [4]:
def pre_process_image(image):        
    # Initialize variables
    left_eye_info = right_eye_info = left_eye_bbox = right_eye_bbox = None

    # Quality Assessment: Check if the image is blurry
    if is_image_blurry(image):
        print("Image is too blurry")
        return None

    dlib_faces = detector(image)
    processed_data = []
    for dlib_face in dlib_faces:
        shape = predictor(image, dlib_face)

        for (i, (start, end)) in enumerate([(36,42), (42,48)]):
            eye_landmarks = [(shape.part(point).x, shape.part(point).y) for point in range(start, end)]
            
            # Skip if eye is closed
            if eye_aspect_ratio(eye_landmarks) < 0.2:
                print("Eye is closed")
                continue

            eye_image, (eye_min_x, eye_min_y, eye_max_x, eye_max_y) = extract_eye_region(image, shape, range(start, end))
            print(eye_max_x, eye_max_y)
            print(eye_min_x, eye_min_y)

            pupil_center, pupil_contour = detect_pupil(eye_image)
            # After detecting the pupil in the cropped eye image:
            if pupil_center:
                # Scale the pupil center coordinates down to the original image size
                pupil_center_original = (pupil_center[0] / 4, pupil_center[1] / 4)
                # Transform these coordinates to the global space of the original image
                pupil_center_global = (int(pupil_center_original[0]) + eye_min_x, int(pupil_center_original[1]) + eye_min_y)
    
                pupil_center_global = tuple(pc.item() if isinstance(pc, np.generic) else pc for pc in pupil_center_global)

                #draw contours
                cv2.drawContours(image, [pupil_contour], -1, (0, 255, 0), 2)
                cv2.circle(image, pupil_center_global, 1, (255, 255, 255), -1)
                cv2.imshow("Eye", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


                bounding_box = (eye_min_x, eye_min_y, eye_max_x - eye_min_x, eye_max_y - eye_min_y)
                bounding_box = tuple(bb.item() if isinstance(bb, np.generic) else bb for bb in bounding_box)

                eye_data = {
                    'eye_position': 'left' if i == 0 else 'right',
                    'pupil_center': pupil_center_global,
                    'bounding_box': bounding_box
                }
                processed_data.append(eye_data)

        # Processed data for each eye
    for eye_data in processed_data:
        if eye_data['eye_position'] == 'left':
            left_eye_info = eye_data['pupil_center']
            left_eye_bbox = eye_data['bounding_box']
        else:
            right_eye_info = eye_data['pupil_center']
            right_eye_bbox = eye_data['bounding_box']
    
    # Check if any eye information was detected
    if left_eye_info is None and right_eye_info is None:
        print("No eye information detected")
        return None

    return processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape
    

In [5]:
image = 'data/WilliamOld/images/eye_head_capture_1702669180.1003327.png'
pre_process_image(cv2.imread(image))
# # Load your image here
# image = cv2.imread(image)
# eye_image = extract_eye_region(image)


315 269
277 243
1529.0
361 269
323 243
40.0


([{'eye_position': 'left',
   'pupil_center': (296, 256),
   'bounding_box': (277, 243, 38, 26)},
  {'eye_position': 'right',
   'pupil_center': (323, 255),
   'bounding_box': (323, 243, 38, 26)}],
 (296, 256),
 (323, 255),
 (277, 243, 38, 26),
 (323, 243, 38, 26),
 <_dlib_pybind11.full_object_detection at 0x145cb661230>)